In [73]:
!pip install "openai-agents[litellm]"

In [74]:
!pip install openai typing sendgrid

In [75]:
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import os
import warnings
warnings.filterwarnings("ignore")
import asyncio

# Step1: ***Agent workflow***

In [76]:
agent1_inst="You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

agent2_inst="You are a humorous, engaging sales agent working for ComplAI.\
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty,engaging cold emails that are likely to get response."

agent3_inst="You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [77]:
from google.colab import userdata

os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"]=userdata.get("OPENAI_API_KEY")

In [78]:
sales_agent1=Agent(
    name="Professional Sales Agent",
    instructions=agent1_inst,
    model="litellm/gemini/gemini-2.5-flash"
)

sales_agent2=Agent(
    name="Engaging Sales Agent",
    instructions=agent2_inst,
    model="litellm/gemini/gemini-2.5-flash"
)

sales_agent3=Agent(
    name="Busy Sales Agent",
    instructions=agent3_inst,
    model="litellm/gemini/gemini-2.5-flash"
)

In [79]:
result=Runner.run_streamed(sales_agent1, "write a cold sales email")
# Above returns a coroutine as it is Async function
async for event in result.stream_events():
  if event.type=="raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
    print(event.data.delta, end="", flush=True)

Subject: Streamlining SOC2 Compliance & Audit Readiness for [Company Name]

Dear [Recipient Name],

I'm reaching out today as I understand that for growing companies like [Company Name], achieving and maintaining SOC2 compliance can be a significant undertaking. It often demands substantial internal resources, time, and can be a complex process to navigate, potentially diverting valuable focus from core product innovation.

At ComplAI, we've developed an AI-powered SaaS tool designed specifically to address these challenges. Our platform streamlines every aspect of SOC2 compliance, from continuous monitoring and evidence collection to proactive gap identification and comprehensive audit preparation.

With ComplAI, organizations can:

*   **Automate** many manual compliance tasks, saving significant time.
*   **Proactively identify** and remediate potential compliance gaps before they become issues.
*   **Generate audit-ready documentation** with confidence and accuracy.
*   **Reduce th

In [80]:
message="write a cold sales email"

with trace("Parallel Cold Emails"):
  results=await asyncio.gather(
      Runner.run(sales_agent1, message),
      Runner.run(sales_agent2, message),
      Runner.run(sales_agent3, message)
  )

  outputs=[result.final_output for result in results]

  for output in outputs:
    print(output, "\n\n")

Subject: Streamlining SOC2 Compliance & Audit Readiness for [Company Name]

Dear [Prospect Name],

Given [Company Name]'s focus on [mention something specific about their company - e.g., "rapid growth in the [industry] sector" or "innovative solutions in [area]"], I imagine maintaining robust security and compliance standards, particularly SOC2, is a critical consideration.

Many high-growth companies find that preparing for and maintaining SOC2 compliance can be a significant drain on internal resources, involving extensive manual evidence collection, complex policy management, and the constant pressure of audit readiness.

At ComplAI, we've developed an AI-powered SaaS platform specifically designed to simplify and accelerate your SOC2 compliance journey. Our tool automates critical compliance workflows, streamlines evidence gathering, and provides intelligent insights to ensure continuous audit readiness, freeing your engineering and security teams to focus on core innovation.

We h

In [81]:
sales_picker=Agent(
    name="sales_picker",
    instructions="you pick the best cold sales email from the given options. \
    Imagine you are a customer and pick the one you are most likely to respond to. \
    Do not give any explanation; reply with the selected email only",
    model="litellm/gemini/gemini-2.5-flash"
)

In [82]:
message=message

with trace("Selection from sales people"):
  results=await asyncio.gather(
      Runner.run(sales_agent1, message),
      Runner.run(sales_agent2, message),
      Runner.run(sales_agent3, message)
  )

  outputs=[result.final_output for result in results]

  emails="Cold sales emails:\n\n".join(outputs)

  best=await Runner.run(sales_picker, emails)

  print(f"Best cold sales email:\n{best.final_output}")

Best cold sales email:
Subject: Your SOC2 Audit's Worst Nightmare (and your team's new best friend) 🤝

Hey [Prospect Name],

I'm going to go out on a limb here and guess that 'SOC2 Audit Prep' isn't exactly at the top of your team's 'Most Thrilling Activities' list, right after axe throwing and competitive napping. Am I right?

Let's be honest, it's often seen as the administrative equivalent of trying to herd a thousand paperwork-loving squirrels through a bureaucratic obstacle course. It sucks up engineering time, causes "fun" (read: stress-induced) sleepless nights, and generally feels like a necessary evil to land those big enterprise deals.

But what if I told you there's a way to tame that beast, *without* sacrificing your engineers' sanity or their precious product development time?

That's where **ComplAI** steps in. We're like the superhero sidekick for your SOC2 journey – if the superhero had an advanced AI brain and a penchant for making compliance surprisingly painless.

Ou

# Step2-&-3: ***tools & interactions***

### Converting func into tool by the help of agent built-in func ***function_tool***

In [83]:
@function_tool
def send_mail(body: str):
  """Send out an email with the given body to all sales prospects"""
  sg=sendgrid.SendGridAPIClient(api_key=userdata.get("SENDGRID_API_KEY"))

  from_email=Email("muasif025@gmail.com")
  to_email=To("as29624041@gmail.com")
  content=Content("text/plain",body)
  mail=Mail(from_email, to_email, "Sales email", content).get()

  response=sg.client.mail.send.post(request_body=mail)

  return {"status":"success"}

In [102]:
send_mail

FunctionTool(name='send_mail', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_mail_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x79072383a340>, strict_json_schema=True, is_enabled=True)

# We can also turn ***Agents*** into ***tools***

In [85]:
tool1=sales_agent1.as_tool(tool_name="sales_agent1", tool_description="write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7907238c19e0>, strict_json_schema=True, is_enabled=True)

In [86]:
# Now we gather all the tools together
description="write a cold sales email"

tool1=sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2=sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3=sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools=[tool1,tool2,tool3, send_mail]

tools


[FunctionTool(name='sales_agent1', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7907238c36a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x790745a13c40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

# Sales Manager - Our planning agent

In [87]:
manager_inst="You are a Sales manager working for ComplAI. \
You use the tools given to you to generate cold sales email. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_mail toolto send the best email (and only the best email) to the user"

sales_manager_agent = Agent(
    name="Sales_Manager",
    instructions=manager_inst,
    tools=tools,
    model="litellm/gemini/gemini-2.5-flash"
)

In [91]:
message="send a cold sales email addressed to 'Dear CEO'"
result = await Runner.run(sales_manager_agent,message)

In [92]:
print(result.final_output)

I have sent the email.


# ***Handoffs***
### handoffs represents a way an agent can delegate to an agent, passing control to it

In [93]:
subject_inst="You can write a subject for a cold sales email. \
You are given a message/email and you need to write a subject for an email that is likely to get a response."

html_inst="You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown and \
you need to convert it to an HTML email body with simple,clear,compelling layout and design."

subject_agent=Agent(
    name="Email Subject Writer",
    instructions=subject_inst,
    model="litellm/gemini/gemini-2.5-flash"
  )
subject_tool=subject_agent.as_tool(tool_name="subject_writer",tool_description="Write a subject for a cold sales email")

html_agent=Agent(
    name="HTML email body converter",
    instructions=html_inst,
    model="litellm/gemini/gemini-2.5-flash"
    )
html_tool=html_agent.as_tool(tool_name="html_converter",tool_description="convert a text email body into an HTML email body")

In [106]:
@function_tool
def send_html_mail(subject: str, html_body:str):
  """Send out an email with given subject and HTML body to all sales prospects"""
  sg=sendgrid.SendGridAPIClient(api_key=userdata.get("SENDGRID_API_KEY"))

  from_email=Email("muasif025@gmail.com")
  to_email=To("as29624041@gmail.com")
  content=Content("text/html",html_body)
  mail=Mail(from_email, to_email, subject, content).get()

  response=sg.client.mail.send.post(request_body=mail)
  return {"status":"success"}

In [95]:
send_html_mail

FunctionTool(name='send_html_mail', description='Send out an email with given subject and HTML body to all sales prospects', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_html_mail_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x79072383bb00>, strict_json_schema=True, is_enabled=True)

In [107]:
tools=[subject_tool, html_tool, send_html_mail]

In [108]:
email_manager_inst="You are an email formatter and sender, you receive the body of an email to be sent. \
you first use the subject_tool tool to write a subject for the email, then use the html_tool tool to convert the email body to HTML. \
Finally,you use the send_html_mail tool to send the email with the subject and HTML body"

emailer_agent=Agent(
    name="Email Manager",
    instructions=email_manager_inst,
    tools=tools,
    model="litellm/gemini/gemini-2.5-flash",
    handoff_description="Convert given text email body to an HTML email body and send it with subject"
)

In [109]:
email_gen_tools=[tool1, tool2, tool3]
handoffs=[emailer_agent]

In [111]:
sales_manager_inst="You are a Sales manager working for ComplAI. \
You use the tools given to you to generate cold sales email. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools at least once before choosing the best one. \
You can use tool multiple times if you are not satisfied with the results from the first try. \
You should select the single best email using your own judgement of which email will be most effective and likely to get response. \
After picking the email, you handoff to the Email Manager Agent to format and send the email."

sales_manager_agent=Agent(
    name="Sales Manager",
    instructions=sales_manager_inst,
    tools=email_gen_tools,
    handoffs=handoffs,
    model="litellm/gemini/gemini-2.5-flash"
)

In [112]:
message="Send out a cold sales email addressed to Dear CEO from Alice"

result = await Runner.run(sales_manager_agent, message)

In [113]:
print(result.final_output)

The email has been sent successfully.
